In [1]:
from atgreen import *
from tqdm import tqdm

In [ ]:
PATH=""
CELL_BUFFER=3000
ROUTING_PROFILE="foot"
OSRM_WORKING_FOLDER="/home/aliceb/gisops_osrm_nodejs"
OSM_INPUT_FOLDER_CITIES=f"{PATH}/osm/cities/"

In [3]:
os.chdir(PATH)
db_params=pickle.load(open(f"{PATH}/dicts/db_params.pickle", 'rb'))
cities_list=query4table('cities_boundary','public', db_params)['city']

### Identify set of pairs for which we have to compute all distances

In [ ]:
#Load grid
for ind, city in enumerate(cities_list):
    print(ind)
    print(city)
    if os.path.exists(f"{PATH}/osm/cities/{city}.osm.pbf"):
        print('OSM file exists.')
        grid=query4grid(city, db_params)
        grid['lat']=grid.geom.centroid.y
        grid['long']=grid.geom.centroid.x

        #As origin cells only keep cells with positive population
        grid_source = grid.query("population > 0")
        #Reproject it to ensure identification of buffered area within which to compute distances
        grid_source=project_gdf(grid_source)
        grid_dest = grid.copy()

        #Identify for each cell, buffered area of 3 km [geodesic], within which to compute distances
        grid_source['geom'] = grid_source.geometry.centroid.buffer(CELL_BUFFER)
        #Reproject to standard CRS 
        grid_source = grid_source.to_crs('epsg:4326')
        grid_dest = grid_dest.to_crs('epsg:4326')
        grid = grid.to_crs('epsg:4326')

        for var in ['lat','long', 'x', 'y']:
            grid_source.rename(columns={var:f"{var}_source"}, inplace=True)
            grid_dest.rename(columns={var:f"{var}_dest"}, inplace=True)
        gdf = grid_source[['lat_source','long_source','geom', 'x_source', 'y_source']].sjoin(grid_dest[['lat_dest','long_dest','geom', 'x_dest', 'y_dest']])


        # First identify non-accessible cells according to heuristic
        #Apply osmium bindings to get streets from the foot profile of OSRM 
        filename=f"{PATH}/osm/cities/{city}.osm.pbf"
        features={'highway':['primary','secondary','tertiary','unclassified',
                            'residential','road','living_street','service','track','path','steps','pedestrian','footway']}

        streets=waysExtraction(filename, features, True, False)

        #Both datasets are in crs EPSG:4326
        accessible=gpd.sjoin(grid, streets, how='left', predicate='intersects')
        accessible_cells=accessible[accessible['osm_key'].isnull()==False][['x', 'y']].drop_duplicates()
        accessible_cells=accessible_cells[['x','y']]
        accessible_cells['walk_access']=1

        #Merge info in original dataset
        gdf=pd.merge(gdf, accessible_cells, left_on=['x_source','y_source'], right_on=['x','y'], how='left')
        gdf=gdf.rename(columns={'walk_access':'walk_access_source'})
        gdf.loc[gdf['walk_access_source'].isnull()==True, 'walk_access_source']=0
        gdf=pd.merge(gdf, accessible_cells, left_on=['x_dest','y_dest'], right_on=['x','y'], how='left')
        gdf=gdf.rename(columns={'walk_access':'walk_access_dest'})
        gdf.loc[gdf['walk_access_dest'].isnull()==True, 'walk_access_dest']=0
        gdf['walk_access']=gdf[['walk_access_source', 'walk_access_dest']].min(axis=1)
        gdf=gdf[['lat_source', 'long_source','x_source', 'y_source','walk_access_source','lat_dest', 'long_dest', 'x_dest', 'y_dest','walk_access_dest','walk_access']]

        ### Compute actual walking distances using osrm
        profile=ROUTING_PROFILE
        working_folder=OSRM_WORKING_FOLDER
        osm_inputfolder=OSM_INPUT_FOLDER_CITIES

        if len(gdf[gdf['walk_access']==1])>0:
            start=datetime.datetime.now()
            print(start)
            res=osrm_files_creation(city, osm_inputfolder, working_folder, profile)
            if res.returncode!=0:
                with open(f"{working_folder}/OSRM_diagnostic.csv", "a") as file:
                    file.write(city+','+'error: file creation'+'\n') 
            else:
                print(datetime.datetime.now())

                all_done=0
                step=0
                gdf['walk_durations']=np.nan
                while all_done==0:
                    print(str(city)+" "+str(step)+", missing pairs:"+str(len(gdf)-len(gdf[gdf['walk_durations'].isnull()==False])))
                    subset=coords_vector_identification(gdf, 2000, f"{city}_coords.csv", working_folder)    #vector size limit of 2000 chosen to optimize computational time
                    res=one_run_osrm(city, f"{city}_coords.csv", f"{city}_durs.csv", working_folder)
                    if res.returncode!=0:
                            with open(f"{working_folder}/OSRM_diagnostic_isglobal_unito_greenaccessibility.csv", "a") as file:
                                file.write(city+','+'error: OSRM run'+'\n') 
                    else:
                        gdf=merge_one_run(gdf, subset,f"{city}_durs.csv", working_folder)
                        step=step+1
                        if len(gdf[(gdf['walk_durations'].isnull()==True) & (gdf['walk_access']==1)])==0:
                            all_done=1
                        os.chdir(path)
                #Transform in minutes
                #Uncomputed distances are stored as string - coerce
                gdf['walk_durations']=pd.to_numeric(gdf['walk_durations'], errors='coerce')
                gdf['walk_minutes']=gdf['walk_durations']/60
                osrm_files_deletion(city, f"{city}_coords.csv", f"{city}_durs.csv" , working_folder)
                gdf[['x_source', 'y_source', 'lat_source', 'long_source','walk_access_source', 'x_dest', 'y_dest','lat_dest', 'long_dest', 'walk_access_dest', 'walk_access', 'walk_minutes']].to_csv(f"{path}distances/{city}.dist.bz2", header=True, index=False, compression='bz2')

            final=datetime.datetime.now()
            print('Report time:')
            print(city)
            print(start)
            print(final)
        else:
            with open(f"{working_folder}/OSRM_diagnostic.csv", "a") as file:
                    file.write(city+','+'error: no cell reachable'+'\n') 
    else:
        with open(f"{working_folder}/OSRM_diagnostic.csv", "a") as file:
            file.write(city+','+'OSM file not found.'+'\n') 